In [1]:
# ! pip install scikit-image
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def evaluate_batch_ssim_psnr(tensor1, tensor2):
    """
    Compute SSIM and PSNR between two batches of RGB images (PyTorch tensors).

    Args:
        tensor1: torch.Tensor, shape (B, 3, H, W), values in [0, 1]
        tensor2: torch.Tensor, shape (B, 3, H, W), values in [0, 1]

    Returns:
        A tuple: (metrics_list, mean_ssim, mean_psnr)
            - metrics_list: list of dicts, each with 'ssim' and 'psnr'
            - mean_ssim: float
            - mean_psnr: float
    """
    assert tensor1.shape == tensor2.shape, "Both tensors must have the same shape"
    assert tensor1.ndim == 4 and tensor1.shape[1] == 3, "Tensors must have shape (B, 3, H, W)"

    results = []
    B = tensor1.shape[0]

    for i in range(B):
        img1 = tensor1[i].detach().cpu().numpy().transpose(1, 2, 0)
        img2 = tensor2[i].detach().cpu().numpy().transpose(1, 2, 0)

        ssim_score = ssim(img1, img2, channel_axis=-1, data_range=1.0)
        psnr_score = psnr(img1, img2, data_range=1.0)

        results.append({'ssim': ssim_score, 'psnr': psnr_score})

    mean_ssim = np.mean([r['ssim'] for r in results])
    mean_psnr = np.mean([r['psnr'] for r in results])

    return results, mean_ssim, mean_psnr

In [2]:
from PIL import Image
import torchvision.transforms as transforms
import torch

**SSIM**
- 完全一样的图片 → SSIM = 1
- 图片有轻微噪声 → SSIM ≈ 0.95+
- 图片模糊或者有强烈失真 → SSIM 可能掉到 0.5甚至更低

**PSNR**
- 完全一样的图片 → PSNR趋于无穷大（理论上）
- 正常的高质量恢复图 → PSNR在30-40 dB
- 差劲的重建 → PSNR可能只有20 dB左右

## fine-tune

In [ ]:
# 定义文件夹路径
actual_dir = "C:/Users/mxx/Desktop/pix2pix_inference/fine_tune"
predicted_dir = "C:/Users/mxx/Desktop/pix2pix_inference/fine_tune"

# 定义transform
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转为 [0,1] 范围的 Tensor
])

# 存放所有图片的list
actual_images = []
predicted_images = []

# 遍历文件夹中的文件，寻找成对的 actual 和 predicted 图片
for filename in os.listdir(actual_dir):
    if filename.startswith("sample_") and filename.endswith("_actual.png"):
        # 提取编号
        idx = filename.split("_")[1]
        actual_path = os.path.join(actual_dir, filename)
        predicted_filename = f"sample_{idx}_predicted.png"
        predicted_path = os.path.join(predicted_dir, predicted_filename)

        # 确保 predicted 图片存在
        if os.path.exists(predicted_path):
            img_actual = transform(Image.open(actual_path).convert('RGB'))
            img_pred = transform(Image.open(predicted_path).convert('RGB'))
            actual_images.append(img_actual)
            predicted_images.append(img_pred)

# 如果读取到了有效图片对
if actual_images and predicted_images:
    # 合并成 batch（shape: B, 3, H, W）
    actual_batch = torch.stack(actual_images, dim=0)
    predicted_batch = torch.stack(predicted_images, dim=0)

    # 调用评估函数
    metrics_list, mean_ssim, mean_psnr = evaluate_batch_ssim_psnr(actual_batch, predicted_batch)

    # 打印结果
    print("Mean SSIM:", mean_ssim)
    print("Mean PSNR:", mean_psnr)
else:
    print("No valid image pairs found.")

Mean SSIM: 0.8629219
Mean PSNR: 21.20329825620129


## baseline

In [ ]:
# 定义文件夹路径
actual_dir = "C:/Users/mxx/Desktop/pix2pix_inference/baseline"
predicted_dir = "C:/Users/mxx/Desktop/pix2pix_inference/baseline"

# 定义transform
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转为 [0,1] 范围的 Tensor
])

# 存放所有图片的list
actual_images = []
predicted_images = []

# 遍历文件夹中的文件，寻找成对的 actual 和 predicted 图片
for filename in os.listdir(actual_dir):
    if filename.startswith("sample_") and filename.endswith("_actual.png"):
        # 提取编号
        idx = filename.split("_")[1]
        actual_path = os.path.join(actual_dir, filename)
        predicted_filename = f"sample_{idx}_predicted.png"
        predicted_path = os.path.join(predicted_dir, predicted_filename)

        # 确保 predicted 图片存在
        if os.path.exists(predicted_path):
            img_actual = transform(Image.open(actual_path).convert('RGB'))
            img_pred = transform(Image.open(predicted_path).convert('RGB'))
            actual_images.append(img_actual)
            predicted_images.append(img_pred)

# 如果读取到了有效图片对
if actual_images and predicted_images:
    # 合并成 batch（shape: B, 3, H, W）
    actual_batch = torch.stack(actual_images, dim=0)
    predicted_batch = torch.stack(predicted_images, dim=0)

    # 调用评估函数
    metrics_list, mean_ssim, mean_psnr = evaluate_batch_ssim_psnr(actual_batch, predicted_batch)

    # 打印结果
    print("Mean SSIM:", mean_ssim)
    print("Mean PSNR:", mean_psnr)
else:
    print("No valid image pairs found.")

Mean SSIM: 0.6161285
Mean PSNR: 11.739744848814643
